<a href="https://colab.research.google.com/github/caua-braga-de-lima/atrial_fibrilation_detection/blob/master/ECG_AF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Este notebook faz uso da base de dados MIT-BIH Atrial Fibrillation Database

http://ecg.mit.edu/george/publications/afib-cinc-1983.pdf

Goldberger, A., Amaral, L., Glass, L., Hausdorff, J., Ivanov, P. C., Mark, R., ... & Stanley, H. E. (2000). PhysioBank, PhysioToolkit, and PhysioNet: Components of a new research resource for complex physiologic signals. Circulation [Online]. 101 (23), pp. e215–e220.

In [1]:
from IPython.display import display
import matplotlib.pyplot as plt

%matplotlib inline
import numpy as np
import os
import shutil
import posixpath

!pip install wfdb
import wfdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 kB 2.0 MB/s eta 0:00:00


#Data Description
Of these, 23 records include the two ECG signals (in the .dat files); records 00735 and 03665 are represented only by the rhythm (.atr) and unaudited beat (.qrs annotation files.)<br>

RECORD ANNOTATIONS:<br>

.atr - rhythm<br>
.qrs - beat (generated by detector)<br>
.qrsc - beat (corrected manually)<br>


In [3]:
import os
import wfdb
import numpy as np
from google.colab import drive

# Montar Google Drive
drive.mount("/content/drive")

# Caminho para a pasta raiz 'record'
root_dir = "/content/drive/Shareddrives/PETDRIVE-CAUB/DETECÇÃO DE FIBRILAÇÃO ATRIAL EM ECG/IMPLEMENTAÇÕES/record"

# Duração do segmento em segundos
segment_duration = 5


# Função para processar um arquivo de ECG
def process_record(record_path):
    """
    Processa um registro de ECG, segmenta em intervalos de 5 segundos e classifica cada segmento com base na presença de FA.

    :param record_path: Caminho do registro de ECG (sem extensão).
    :return: Uma lista de segmentos e uma lista de rótulos.
    """
    # Ler o registro e anotações
    record = wfdb.rdrecord(record_path)
    annotation = wfdb.rdann(record_path, "atr")

    # Frequência de amostragem
    fs = record.fs

    # Número de amostras por segmento
    samples_per_segment = int(segment_duration * fs)

    # Lista para armazenar segmentos e rótulos
    segments = []
    labels = []

    # Percorrer os segmentos
    num_segments = len(record.p_signal) // samples_per_segment
    for i in range(num_segments):
        start = i * samples_per_segment
        end = start + samples_per_segment
        segment = record.p_signal[start:end]

        # Verificar se há anotações de fibrilação atrial neste segmento
        ann_in_segment = [a for a in annotation.sample if start <= a < end]
        label = (
            1
            if any(
                annotation.symbol[j] == "AFIB"
                for j, a in enumerate(annotation.sample)
                if start <= a < end
            )
            else 0
        )

        segments.append(segment)
        labels.append(label)

    return segments, labels


# Processar todos os registros
all_segments = []
all_labels = []

for subdir in os.listdir(root_dir):
    subdir_path = os.path.join(root_dir, subdir)
    if os.path.isdir(subdir_path):
        for file in os.listdir(subdir_path):
            if file.endswith(".dat"):
                record_path = os.path.join(subdir_path, file[:-4])
                segments, labels = process_record(record_path)
                all_segments.extend(segments)
                all_labels.extend(labels)

# Exibir o número de segmentos processados e distribuição de rótulos
print(f"Total de segmentos: {len(all_segments)}")
print(f"Distribuição de rótulos: {np.bincount(all_labels)}")

# Salvar os segmentos e rótulos em um arquivo, se necessário
# np.save('segments.npy', all_segments)
# np.save('labels.npy', all_labels)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/Shareddrives/PETDRIVE-CAUB/DETECÇÃO DE FIBRILAÇÃO ATRIAL EM ECG/IMPLEMENTAÇÕES/record/record3/04015.dat'